In [3]:
from rdkit import Chem
import numpy as np

In [10]:

def rdk_node_features_map():
    base_map = {"Idx":0,"AtomicNum":1,"FormalCharge":2,"NumExplicitHs":3,"IsAromatic":4}
    base_map = {**base_map,**rdk_enum_type_to_map(Chem.ChiralType,len(base_map))}
    return {**base_map,**rdk_enum_type_to_map(Chem.HybridizationType,len(base_map))}

def rdk_edge_features_map():
    base_map = {"Weight":0,"Conjugated":1}
    base_map = {**base_map,**rdk_enum_type_to_map(Chem.BondType,len(base_map))}
    base_map = {**base_map,**rdk_enum_type_to_map(Chem.BondDir,len(base_map))}
    return {**base_map,**rdk_enum_type_to_map(Chem.BondStereo,len(base_map))}

def rdk_enum_type_to_map(enum_type,offset=0):
    enum_map={}
    for val in enum_type.names:
        enum_map[enum_type.names[val].name]=enum_type.names[val].real+offset
    return enum_map

def rdk_enum_val_to_one_hot(enum_type):
    one_hot_vec = np.zeros((1,len(enum_type.values)))
    one_hot_vec[0,enum_type.real]=1
    return one_hot_vec


In [20]:
mol = Chem.MolFromSmiles('Cc1ccccc1')

n_map= rdk_node_features_map()
e_map = rdk_edge_features_map()
atms = mol.GetAtoms()
bnds = mol.GetBonds()
n = len(atms)
A = np.zeros((n,n))
X = np.zeros((n,len(n_map)))
W = np.zeros((2*len(bnds),len(e_map)))

for atom in atms:
    i = atom.GetIdx()
    X[i,n_map['Idx']]=i
    X[i,n_map['AtomicNum']]=atom.GetAtomicNum()
    X[i,n_map['FormalCharge']]=atom.GetFormalCharge()
    X[i,n_map['NumExplicitHs']]=atom.GetNumExplicitHs()
    X[i,n_map['IsAromatic']]=int(atom.GetIsAromatic() == True)
    X[i,n_map[atom.GetChiralTag().name]]=1 #atom.GetChiralTag().real
    X[i,n_map[atom.GetHybridization().name]]= 1 #atom.GetHybridization().real

#  base_map = {"Weight":0,"Conjugated":1}
p=0
_p=n
for bond in bnds:
    print(bond)
    A[bond.GetBeginAtomIdx(),bond.GetEndAtomIdx()]=1
    A[bond.GetEndAtomIdx(),bond.GetBeginAtomIdx()]=1
    '''p = n*bond.GetBeginAtomIdx()+bond.GetEndAtomIdx()+1
    _p = n*bond.GetEndAtomIdx()+bond.GetBeginAtomIdx()+1
    print(bond.GetBeginAtomIdx(),bond.GetEndAtomIdx(),p,_p)'''
    W[p,e_map['Weight']]=1
    W[_p,e_map['Weight']]=1

    W[p,e_map['Conjugated']]=int(bond.GetIsConjugated() == True)
    W[_p,e_map['Conjugated']]=int(bond.GetIsConjugated() == True)

    W[p,e_map[bond.GetBondType().name]]= 1 #bond.GetBondType().real
    W[_p,e_map[bond.GetBondType().name]]= 1 #bond.GetBondType().real

    W[p,e_map[bond.GetBondDir().name]]= 1 
    W[_p,e_map[bond.GetBondDir().name]]= 1

    W[p,e_map[bond.GetStereo().name]]= 1 
    W[_p,e_map[bond.GetStereo().name]]= 1
    p+=1
    _p+=1

In [4]:
import torch 
import numpy as np

X = torch.rand((10,64,5,5,5))

l = np.ones(len(X.shape), dtype=int)
l[0] = X.shape[0]

R = torch.repeat_interleave(X, X.shape[0], dim=0)
L = X.repeat(tuple(l))
m2 = torch.cat((L,R),dim=1)
m2.shape


torch.Size([100, 128, 5, 5, 5])